In [1]:
#!/usr/bin/python

# Import required modules
import os
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mpltkr
import matplotlib.colors as mplcolors
import matplotlib.patches as mplpatches
import matplotlib.lines as mpllines
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Utilities
import project_utils as utils
import h5py

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
color_list = cosmostyle.color_cycle

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/helfrech/.config/matplotlib/stylelib/cosmoLarge.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning:


The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.



# PCA on structures

In [2]:
deem_dir = '../../Processed_Data/DEEM_330k'
iza_dir = '../../Processed_Data/IZA_226'
data_dir = 'Data/6.0/Linear_Models/PCA'

In [3]:
# Load Deem 10k PCA
deem_10k_idxs = np.loadtxt('../../Processed_Data/DEEM_330k/deem_10k.idxs', dtype=int)
deem_pca = utils.load_hdf5(f'{deem_dir}/{data_dir}/pca_structures.hdf5', 
                           datasets=None, indices=deem_10k_idxs, concatenate=True)

# Load IZA PCA
iza_pca = utils.load_hdf5(f'{iza_dir}/{data_dir}/pca_structures.hdf5', 
                          datasets=None, indices=None, concatenate=True)
cantons = np.loadtxt('../../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)

In [5]:
# 3D scatter
fig = make_subplots(rows=3, cols=3,
                    column_widths=[1.0, 1.0, 1.0],
                    row_heights=[1.0, 1.0, 1.0],
                    specs=)

# Deem
fig.add_trace(go.Scatter3d(x=deem_pca[:, 0], 
                           y=deem_pca[:, 1], 
                           z=deem_pca[:, 2],
                           mode='markers', 
                           marker=dict(color=color_list[9], size=1),
                           name='DEEM'))
# IZA
for i in range(1, np.amax(cantons) + 1):
    iza_pts = np.nonzero(cantons == i)[0]
    fig.add_trace(go.Scatter3d(x=iza_pca[iza_pts, 0], 
                               y=iza_pca[iza_pts, 1], 
                               z=iza_pca[iza_pts, 2],
                               mode='markers', 
                               marker=dict(color=color_list[i], size=2),
                               name=f'IZA{i}'))

fig.update_layout(template='plotly_white',
                  font=dict(size=16),
                  scene=dict(xaxis_title='PC<sub>1</sub>',
                             yaxis_title='PC<sub>2</sub>',
                             zaxis_title='PC<sub>3</sub>'),
                  legend=dict(x=1.0, y=1.0,
                              xanchor='left', yanchor='top',
                              itemsizing='constant'),
                  autosize=False,
                  width=800, height=800)

fig.show()